# Set up model and hyperparameters

In [1]:
from models.WaveNetVAE.WaveVae import WaveNetVAE
from models.WaveNetVAE.WVData import WVDataset
import torch
from torch.utils.data import DataLoader
import warnings
warnings.filterwarnings("ignore")

"""
Hyperparameters
"""

learning_rate = 0.000001
batchsize = 4
device='cuda:2'
input_size = (40, 112)
upsamples = [2, 2, 2, 2, 2, 2, 2, 2]
zsize = 32

WaveVAE = WaveNetVAE(input_size,
                     num_hiddens = 768,
                     upsamples = upsamples,
                     zsize = zsize,
                    out_channels = 1)

WaveVAE.to(device)

# VAEDataset = WVDataset(audio_path = "../ConvDenoiser_Old/clips",
#                        length = 4096,
#                        skip_size = 4096 // 2,
#                        sample_rate = 24000,
#                        max_files = 50,
#                        hop_length = 128)

# val_VAEDataset = WVDataset(audio_path = "../ConvDenoiser_Old/testdata",
#                        length = 4096,
#                        skip_size = 4096 // 2,
#                        sample_rate = 24000,
#                        max_files = 200,
#                        hop_length = 128)

# VAEDataloader = DataLoader(VAEDataset,
#                            batch_size = batchsize,
#                            shuffle = True)

# val_VAEDataloader = DataLoader(val_VAEDataset,
#                            batch_size = batchsize,
#                            shuffle = False)

WaveNet Receptive Field:  4093


WaveNetVAE(
  (encoder): Encoder(
    (ReL): ReLU()
    (conv_1): Conv1d(40, 768, kernel_size=(3,), stride=(1,), padding=same)
    (conv_2): Conv1d(40, 768, kernel_size=(3,), stride=(1,), padding=same)
    (downsample): Conv1d(768, 768, kernel_size=(4,), stride=(2,), padding=(1,))
    (resblocks): ModuleList(
      (0): Conv1d(768, 768, kernel_size=(3,), stride=(1,), padding=same)
      (1): Conv1d(768, 768, kernel_size=(3,), stride=(1,), padding=same)
    )
    (relublocks): ModuleList(
      (0): Sequential(
        (0): Conv1d(768, 768, kernel_size=(3,), stride=(1,), padding=same)
        (1): ReLU(inplace=True)
        (2): Conv1d(768, 768, kernel_size=(3,), stride=(1,), padding=same)
        (3): ReLU(inplace=True)
      )
      (1): Sequential(
        (0): Conv1d(768, 768, kernel_size=(3,), stride=(1,), padding=same)
        (1): ReLU(inplace=True)
        (2): Conv1d(768, 768, kernel_size=(3,), stride=(1,), padding=same)
        (3): ReLU(inplace=True)
      )
      (2): Sequen

# Test tensor

In [12]:
import torchaudio

loss_fn = torch.nn.MSELoss()
onehot, mfcc, target = next(iter(VAEDataloader))
onehot = onehot.to(device)
mfcc = mfcc.to(device)
target = target.to(device)

print("Trying tensors with sizes:")
print("Onehot size: ", onehot.size(), "| MFCC size: ", mfcc.size(), "| Target size: ", target.size())

output, _, _ = WaveVAE(onehot, mfcc, True)
print("Tensors passed through model succesfully", output.size())

reconstruction_loss = loss_fn(output[:, :, -1], target)
print("Reconstruction loss =", reconstruction_loss.item())

# onehot = torch.nn.functional.sigmoid(onehot[0, :, -1])
# output = torch.nn.functional.sigmoid(output[0, :, -1])
# onehotmax = torch.argmax(onehot)
# outputmax = torch.argmax(output)

print(onehot[0, :, -1], output[0, :, -1], target[0])




Trying tensors with sizes:
Onehot size:  torch.Size([8, 1, 4096]) | MFCC size:  torch.Size([8, 40, 33]) | Target size:  torch.Size([8])
Tensors passed through model succesfully torch.Size([8, 1, 4096])
Reconstruction loss = 0.021551592275500298
tensor([-0.0264], device='cuda:0') tensor([0.0110], device='cuda:0', grad_fn=<SelectBackward0>) tensor(-0.0238, device='cuda:0')


# Start training

In [2]:
from models.WaveNetVAE.train import train
from torch.utils.tensorboard import SummaryWriter
import warnings
warnings.filterwarnings("ignore")
writer = SummaryWriter()

train(WaveVAE, VAEDataloader, val_VAEDataloader, 
      writer = writer, 
      learning_rate = learning_rate,
      epoch_amount = 100,
      logs_per_epoch = 5,
      kl_anneal = 0.01,
      max_kl = 0.5,
      device = device)


Training. Epoch: 0. Loss for step 1: n.v.t.:   0%|          | 0/522 [00:00<?, ?it/s]

Validating:   0%|          | 0/384 [00:00<?, ?it/s]

Validating:   0%|          | 0/384 [00:00<?, ?it/s]

Validating:   0%|          | 0/384 [00:00<?, ?it/s]

Validating:   0%|          | 0/384 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [5]:
torch.save(WaveVAE.state_dict(), './modeltest')

# Generate Audio

In [ ]:
val_VAEDataset = WVDataset(audio_path = "../ConvDenoiser_Old/testdata",
                       length = 4096,
                       skip_size = 1,
                       sample_rate = 24000,
                       max_files = 1,
                       hop_length = 128)

val_VAEDataloader = DataLoader(val_VAEDataset,
                           batch_size = 1,
                           shuffle = False)
with torch.no_grad():
    generated_audio = WaveVAE.inference(val_VAEDataloader, size = 4096, device = device)

Loading and preprocessing files to dataset.:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/24000 [00:00<?, ?it/s]

torch.Size([1, 1, 4096])


  0%|          | 0/24000 [00:00<?, ?it/s]

In [2]:
WaveVAE.load_state_dict(torch.load('./modeltest'))

<All keys matched successfully>

In [8]:
import IPython
print(generated_audio.size())
IPython.display.display(IPython.display.Audio(generated_audio.squeeze().detach().cpu().numpy() * 2 - 1, rate = 24000))

torch.Size([1, 1, 99839])
